# HW6_97110069

### 1)
Explanations for risk based and behavioural stand points have been highlighted in 'Explanations for the Momentum Premium.pdf'.

### 2)
Items A to C from page 708 to 711 are the reasons we are looking at post-ranking returns. I have highlighted reasons in green.

In [117]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

In [48]:
def read_data():
    market_cap = pd.read_excel('monthly market cap panel.xlsx')
    market_cap.set_index('idx')
    market_cap = market_cap.dropna(axis=1, how='all')
    returns = pd.read_excel('monthly adj_returns panel.xlsx')
    returns.set_index('idx')
    returns = returns.dropna(axis=1, how='all')
    return market_cap , returns

market_cap , returns = read_data()
print(len(returns.columns))

378


In [49]:
def get_small_market_caps():
    x = market_cap.mean()
    small_market_caps = x[x < x.quantile(0.1)]
    small_market_cap_indexes = small_market_caps.index
    return small_market_cap_indexes

small_mcap_stocks = get_small_market_caps()
returns = returns.drop(columns = small_mcap_stocks,axis = 1)

In [202]:
class MomentumStrategy:
    def __init__(self, rank_period, hold_period, returns):
        self.rank_period = rank_period
        self.hold_period = hold_period
        self.returns = returns
        self.get_cumprod_returns()
        self.holding_returns = None
        self.ranking_returns = None
        self.get_returns_for_rank_and_hold()
        self.positions = []
        self.portfolio_return = [0] * (len(self.returns) + 20)
        
    def get_cumprod_returns(self):
        self.returns.fillna(1, inplace = True)
        self.returns = self.returns.cumprod()
        
    def get_returns_for_rank_and_hold(self):
        
        self.ranking_returns = self.returns.div(self.returns.shift(self.rank_period))
        self.ranking_returns -= 1
#         self.holding_returns = self.returns.shift(-self.hold_period).div(self.returns)
    
    def get_portfolio_in_time(self):
        for index in self.returns.index:
#             TODO : 3 -> len(df)//10
            highest_return_stocks = self.ranking_returns.loc[index].nlargest(len(self.returns)//10).to_frame().T.columns
#             print(highest_return_stocks)
            if len(highest_return_stocks) > 0:
                position = Position(self.ranking_returns,index, highest_return_stocks, self.hold_period)
                position.get_position_return()
                self.positions.append(position)
    
    def get_return_in_time(self):
        for position in self.positions:
            for i in range(position.index, position.index + self.hold_period):
                try:
                    j = 0
                    if len(position.position_returns) > j:
                        self.portfolio_return[i] += position.position_returns[j]
                    j += 1
                except Exception as e:
                    print(position.position_returns)
                    
                    
#         print('all returns are:', self.portfolio_return)


class Position:
    def __init__(self, returns, index, stocks, holding_period):
#         print('stocks',stocks)
        self.returns = returns.loc[index:index + holding_period - 1,stocks]
#         print('returns',self.returns)
        self.hold_period = holding_period
        self.index = index
        self.num_of_stocks = len(stocks)
        self.position_returns = [0] * holding_period
        
    
    def get_position_return(self):
        for column in self.returns.columns:
            self.returns[column] /= self.num_of_stocks
            return_of_stock = self.returns[column].to_list()
            
            for i in range(len(return_of_stock)):
#                 print(i)
                self.position_returns[i] += return_of_stock[i]  
            
            
        
        
        



In [203]:
m = MomentumStrategy(6 , 5 , returns.copy(deep = True))
m.get_portfolio_in_time()
m.get_return_in_time()

In [210]:
l = np.asarray(m.portfolio_return)
l = l[l != 0]


In [218]:
for i in range(1, 5):
    for j in range(1 , 5):
        m = MomentumStrategy(i , j , returns.copy(deep = True))
        m.get_portfolio_in_time()
        m.get_return_in_time()
        l = np.asarray(m.portfolio_return)
        l = l[l != 0]
        print(i , j ,np.mean(l[:-10]))

1 1 0.06430895981936655
1 2 0.12478696117873347
1 3 0.2071189835888003
1 4 0.2352717988588011
2 1 0.12004353499894987
2 2 0.140277033745137
2 3 0.15465232354771014
2 4 0.16414415456112416
3 1 0.08631318261570081
3 2 0.09674540694781066
3 3 0.10007487185255182
3 4 0.09889831421689818
4 1 0.07921789088514765
4 2 0.08570452121193727
4 3 0.08576227761434685
4 4 0.07832179813981151
